## RUN

### First test on endomondo_sample db

### REMOVE DUPES.

#### Count the number of dupes

```select count(*) from (select count(*) from run group by altitude, heart_rate, latitude, longitude, speed, workoutid, time having count(*) > 1) as foo;```
```
 count 
-------
  6815
```

#### Remove the dupes

```ALTER TABLE run ADD COLUMN id SERIAL PRIMARY KEY;
DELETE FROM run
WHERE id IN (SELECT id
              FROM (SELECT id,
                             ROW_NUMBER() OVER (partition BY altitude, heart_rate, latitude, longitude, speed, workoutid, time ORDER BY id) AS rnum
                     FROM run) t
              WHERE t.rnum > 1);
ALTER TABLE run drop column id;```
```
DELETE 6825
```

```select count(*) from (select count(*) from run group by altitude, heart_rate, latitude, longitude, speed, workoutid, time having count(*) > 1) as foo;```

```
 count 
-------
     0
```

### Vacuum
```vacuum full run;```

## Speed

### Value given in kph. Range should be from 0-50.

```
select * from histogram('speed', 'run');
```
```
 bucket |              range              |  freq   |       bar       
--------+---------------------------------+---------+-----------------
      1 | [0.0000000000,11.7612000000]    | 1203608 | ***************
      2 | [11.7643000000,23.5224000000]   |  944340 | ************
      3 | [23.5260000000,35.2836000000]   |   13137 | 
      4 | [35.2944000000,47.0340000000]   |    2158 | 
      5 | [47.0700000000,58.7736000000]   |     309 | 
      6 | [58.8708000000,70.2720000000]   |      80 | 
      7 | [70.7508000000,82.3320000000]   |      37 | 
      8 | [83.9268000000,93.6936000000]   |       8 | 
      9 | [94.3164000000,105.1380000000]  |      21 | 
     10 | [106.4340000000,114.8900000000] |      10 | 
     11 | [118.4800000000,127.8470000000] |      10 | 
     12 | [129.4560000000,135.4390000000] |       6 | 
     13 | [142.2830000000,148.9000000000] |       6 | 
     14 | [154.7420000000,162.7090000000] |       9 | 
     15 | [168.3970000000,174.4020000000] |       4 | 
     16 | [180.0000000000,187.0700000000] |       8 | 
     17 | [188.2400000000,199.3540000000] |       5 | 
     18 | [200.3940000000,206.6800000000] |       4 | 
     20 | [224.3810000000,235.1920000000] |       7 | 
     21 | [235.2560000000,235.2560000000] |       1 | 
     ```

#### Simply removing data has implications, there is a relatively small amount of outliers so we will just smooth them.

#### Lets find the average speed and smooth them to that.

```SELECT avg(speed) AS average FROM run where speed < 50 and speed > 0;```
```
       average       
---------------------
 11.6935489411286696
```

### Still too many > 50kph (30mph, no one that fast).

```
update run set speed = 11.7 where speed > 50;
```
```
UPDATE 397
```

```
select * from histogram('speed', 'run');
```
```
 bucket |             range             |  freq  |       bar       
--------+-------------------------------+--------+-----------------
      1 | [0.0000000000,2.4984000000]   |  57989 | *
      2 | [2.5020000000,4.9968000000]   |  15587 | 
      3 | [5.0000000000,7.4952000000]   |  58139 | *
      4 | [7.4988000000,9.9939500000]   | 372444 | ******
      5 | [9.9972000000,12.4920000000]  | 992467 | ***************
      6 | [12.4956000000,14.9904000000] | 523424 | ********
      7 | [14.9940000000,17.4924000000] | 106273 | **
      8 | [17.4960000000,19.9908000000] |  16649 | 
      9 | [19.9944000000,22.4892000000] |   4079 | 
     10 | [22.4964000000,24.9876000000] |   3653 | 
     11 | [24.9912000000,27.4860000000] |   3259 | 
     12 | [27.4896000000,29.9844000000] |   2840 | 
     13 | [29.9880000000,32.4828000000] |   2648 | 
     14 | [32.4864000000,34.9812000000] |   1876 | 
     15 | [34.9884000000,37.4796000000] |   1024 | 
     16 | [37.4868000000,39.9816000000] |    528 | 
     17 | [39.9888000000,42.4656000000] |    373 | 
     18 | [42.4908000000,44.9784000000] |    242 | 
     19 | [45.0000000000,47.4696000000] |    165 | 
     20 | [47.5020000000,49.8888000000] |    108 | 
     21 | [49.9788000000,49.9788000000] |      1 | 
```

#### Much better lets generate first derivative of speed.

```
with dev_list as (
    select round((speed_difference / time_difference),5) as deriv,
           time,
           workoutid
           from (
        select speed_difference,
               case when time_difference = 0 then 1 else time_difference end as time_difference,
               time,
               workoutid
               from (
            select speed - lag(speed) over (partition by workoutid order by time) as speed_difference,
                   time - lag(time) over (partition by workoutid order by time) as time_difference,
                   speed,
                   time,
                   workoutid
                from run order by time)
        as foo)
    as bar
    order by workoutid,
             time )
update run r1
  set speed_first = d1.deriv
  from dev_list as d1
  where d1.workoutid = r1.workoutid and
        d1.time = r1.time;
```

### Sanity check values
```
select * from histogram('speed_first', 'run');
```
```
 bucket |         range         |  freq   |       bar       
--------+-----------------------+---------+-----------------
      1 | [-38.04840,-35.89200] |       3 | 
      2 | [-33.44760,-32.11560] |       3 | 
      3 | [-28.97640,-27.03960] |       2 | 
      4 | [-25.54200,-22.20480] |       5 | 
      5 | [-21.38760,-17.87040] |       5 | 
      6 | [-17.40240,-14.40480] |       7 | 
      7 | [-12.90960,-9.96660]  |      28 | 
      8 | [-9.67680,-5.66520]   |     126 | 
      9 | [-5.65080,-1.61640]   |    1698 | 
     10 | [-1.61460,2.43216]    | 2156648 | ***************
     11 | [2.43360,6.45840]     |     773 | 
     12 | [6.51600,10.47960]    |      77 | 
     13 | [10.64880,14.54760]   |      22 | 
     14 | [14.83200,18.14040]   |      13 | 
     15 | [19.47240,21.57840]   |       2 | 
     16 | [23.77440,26.57160]   |       3 | 
     17 | [27.78120,30.26520]   |       4 | 
     18 | [31.41000,34.05960]   |       7 | 
     19 | [36.64440,37.21680]   |       2 | 
     21 | [42.91560,42.91560]   |       1 | 
```

#### (Re)Generate moving averages

##### Over 50 points
```
with dev_list as (
	select time, 
	       workoutid, 
	       avg(speed) over (partition by workoutid order by time rows between 50 preceding and current row) as mavg 
	from run 
	order by time
)
update run r1 set speed_ma_50 = d1.mavg from dev_list as d1 where d1.workoutid = r1.workoutid and d1.time = r1.time;
```

```
select * from histogram('speed_ma_50', 'run');
```
```
 bucket |        range        |  freq   |       bar       
--------+---------------------+---------+-----------------
      1 | [0.00000,2.48673]   |   54743 | *
      2 | [2.48940,4.97393]   |    7355 | 
      3 | [4.97506,7.46224]   |   46063 | *
      4 | [7.46235,9.94967]   |  368327 | *****
      5 | [9.94968,12.43708]  | 1054193 | ***************
      6 | [12.43710,14.92447] |  519880 | *******
      7 | [14.92454,17.41186] |   85668 | *
      8 | [17.41214,19.89926] |    7786 | 
      9 | [19.90101,22.38543] |    3434 | 
     10 | [22.38784,24.87388] |    4113 | 
     11 | [24.87459,27.35965] |    3832 | 
     12 | [27.36296,29.84856] |    3400 | 
     13 | [29.85085,32.33576] |    2587 | 
     14 | [32.33809,34.82132] |    1672 | 
     15 | [34.82598,37.30927] |     460 | 
     16 | [37.31428,39.77160] |     141 | 
     17 | [39.80520,42.09755] |      88 | 
     18 | [42.45840,44.17200] |      10 | 
     19 | [44.79120,46.27080] |       8 | 
     20 | [47.69280,49.17600] |       7 | 
     21 | [49.74840,49.74840] |       1 | 
     ```

##### Over 100 points
```
with dev_list as (
	select time, 
	       workoutid, 
	       avg(speed) over (partition by workoutid order by time rows between 100 preceding and current row) as mavg 
	from run 
	order by time
)
update run r1 set speed_ma_100 = d1.mavg from dev_list as d1 where d1.workoutid = r1.workoutid and d1.time = r1.time;
```

```
select * from histogram('speed_ma_100', 'run');
```
```
 bucket |        range        |  freq   |       bar       
--------+---------------------+---------+-----------------
      1 | [0.00000,2.48673]   |   54618 | *
      2 | [2.48940,4.97340]   |    6809 | 
      3 | [4.97485,7.46223]   |   43132 | *
      4 | [7.46227,9.94967]   |  374203 | *****
      5 | [9.94968,12.43707]  | 1067772 | ***************
      6 | [12.43710,14.92446] |  509834 | *******
      7 | [14.92453,17.41176] |   80230 | *
      8 | [17.41227,19.89926] |    7607 | 
      9 | [19.89962,22.38548] |    3275 | 
     10 | [22.38759,24.87386] |    4241 | 
     11 | [24.87586,27.36061] |    4220 | 
     12 | [27.36177,29.84899] |    3553 | 
     13 | [29.85070,32.33630] |    2370 | 
     14 | [32.33691,34.82166] |    1506 | 
     15 | [34.82911,37.28809] |     228 | 
     16 | [37.31940,39.75929] |     109 | 
     17 | [39.80520,41.93040] |      35 | 
     18 | [42.45840,44.17200] |      10 | 
     19 | [44.79120,46.27080] |       8 | 
     20 | [47.69280,49.17600] |       7 | 
     21 | [49.74840,49.74840] |       1 | 
```

## Altitude

### Big range of altitudes, some too low, some too high


```
select * from histogram('altitude', 'run');
```
```
 bucket |                range                |  freq   |       bar       
--------+-------------------------------------+---------+-----------------
      1 | [-500.0000000000,155.2270000000]    | 6220826 | ***************
      2 | [155.2350000000,810.4000000000]     | 2305352 | ******
      3 | [810.4600000000,1465.6700000000]    |  183473 | 
      4 | [1465.6900000000,2120.8000000000]   |   98096 | 
      5 | [2121.0000000000,2775.8000000000]   |   19153 | 
      6 | [2776.4000000000,3426.4000000000]   |    3219 | 
      7 | [3434.0000000000,4086.0000000000]   |    2320 | 
      8 | [4087.8000000000,4740.8000000000]   |     747 | 
      9 | [4751.8000000000,5397.0000000000]   |    1687 | 
     10 | [5399.4000000000,6052.2000000000]   |    1639 | 
     11 | [6054.6000000000,6705.0000000000]   |    3170 | 
     12 | [6708.8000000000,7346.8000000000]   |   28958 | 
     13 | [7429.4000000000,8014.8000000000]   |     533 | 
     14 | [8018.6000000000,8669.2000000000]   |     627 | 
     15 | [8674.2000000000,9324.0000000000]   |    1614 | 
     16 | [9330.6000000000,9975.6000000000]   |    1109 | 
     17 | [9984.2000000000,10638.4000000000]  |    1103 | 
     18 | [10646.6000000000,11292.2000000000] |    1441 | 
     19 | [11294.4000000000,11940.6000000000] |    1475 | 
     20 | [11955.6000000000,12603.0000000000] |     372 | 
     21 | [12604.6000000000,12604.6000000000] |       1 | 
```

#### Lets see how many rows are more thant two standard deviations  from the mean

```
select count(*) from (
	with dev_list as ( 
		select avg(altitude), stddev(altitude), workoutid from run group by workoutid )
select altitude, r1.workoutid 
  from run r1 
  join dev_list d1 on (d1.workoutid = r1.workoutid) 
  where r1.altitude < d1.avg - d1.stddev * 2 or r1.altitude > d1.avg + d1.stddev * 2) 
as foo;
```

##### 323,310 / 12,493,175 => ~2.5%
##### We should just smooth them to be two standard deviations from the mean

##### Fix values too low
```
with dev_list as ( 
	select avg(altitude), stddev(altitude), workoutid from run group by workoutid )
update run as r1 
  set altitude = d1.avg - (d1.stddev * 2) 
  from dev_list as d1 
  where d1.workoutid = r1.workoutid and r1.altitude < d1.avg - (d1.stddev * 2);
  ```
```
UPDATE 113289
```

##### Fix values too high
```
with dev_list as ( 
	select avg(altitude), stddev(altitude), workoutid from run group by workoutid )
update run as r1 
  set altitude = d1.avg + (d1.stddev * 2) 
  from dev_list as d1 
  where d1.workoutid = r1.workoutid and r1.altitude > d1.avg + (d1.stddev * 2);
```
```
UPDATE 218677
```

#### Sanity check

```
select * from histogram('altitude', 'run');
```
```
 bucket |                range                |  freq   |       bar       
--------+-------------------------------------+---------+-----------------
      1 | [-500.0000000000,155.1390000000]    | 6221393 | ***************
      2 | [155.1600000000,810.2591460450]     | 2304852 | ******
      3 | [810.3000000000,1465.4000000000]    |  183303 | 
      4 | [1465.4700000000,2120.5000000000]   |   98099 | 
      5 | [2120.6000000000,2775.4600000000]   |   19177 | 
      6 | [2776.4000000000,3426.4000000000]   |    3187 | 
      7 | [3434.0000000000,4084.4000000000]   |    2369 | 
      8 | [4087.8000000000,4717.7463628016]   |     728 | 
      9 | [4802.8000000000,5394.4000000000]   |    1665 | 
     10 | [5397.0000000000,6050.2000000000]   |    1666 | 
     11 | [6052.2000000000,6705.0000000000]   |    3240 | 
     12 | [6708.8000000000,7349.0065395790]   |   28984 | 
     13 | [7376.4751463155,8014.8000000000]   |     536 | 
     14 | [8018.6000000000,8669.2000000000]   |     619 | 
     15 | [8674.2000000000,9324.0000000000]   |    1617 | 
     16 | [9330.6000000000,9975.6000000000]   |    1107 | 
     17 | [9984.2000000000,10630.2000000000]  |    1094 | 
     18 | [10638.4000000000,11292.2000000000] |    1439 | 
     19 | [11294.4000000000,11940.6000000000] |    1479 | 
     20 | [11955.6000000000,12602.6000000000] |     360 | 
     21 | [12603.0000000000,12603.0000000000] |       1 | 
```

##### Everest is at 8,850 meters, and the most extreme high altitude ultra marathon only gets up to 5,300 meters. The lowest below sea level land is -413 meters. Probably a lot of errant sensor data

##### Lets look at workouts that include altitudes greater than 5300
```
select altitude, time, latitude, longitude, workoutid from run where workoutid in (select distinct(workoutid) from run where altitude > 5300) order by workoutid, time;
```

###### Small sample, look up lat / lng and check altitude. Spot check lat / lngs looks like the data is bad. 
```
     altitude     |    time    |   latitude    |   longitude    | workoutid 
------------------+------------+---------------+----------------+-----------
  4717.7463628016 | 1359891816 | 54.5475614000 |  -1.9052789000 | 156382546
  6892.8000000000 | 1359891819 | 54.5476397000 |  -1.9052535000 | 156382546
  6886.0000000000 | 1359891823 | 54.5477357000 |  -1.9052153000 | 156382546
  6883.4000000000 | 1359891825 | 54.5477830000 |  -1.9052311000 | 156382546
  6879.2000000000 | 1359891828 | 54.5478676000 |  -1.9052446000 | 156382546
  6875.0000000000 | 1359891831 | 54.5479189000 |  -1.9052517000 | 156382546
  6872.4000000000 | 1359891833 | 54.5479453000 |  -1.9052918000 | 156382546
  6868.2000000000 | 1359891836 | 54.5479799000 |  -1.9053959000 | 156382546
```

###### How many workouts is it?
```
select count(*) from  (select distinct(workoutid) from run where altitude > 5300) as foo;
```

###### 87 / 25,000.
```
select count(*) from run where altitude > 5300;
```
##### 42,172  probably should just drop the altitude for those workouts, as that is ~484 per workout.
```
update run set altitude = NULL where workoutid in (select distinct(workoutid) from run where altitude > 5300);
```


##### (Re)Generate first derivative
```
with dev_list as (
    select round((alt_difference / time_difference),5) as deriv, time, workoutid from ( 
        select alt_difference, case when time_difference = 0 then 1 else time_difference end as time_difference, altitude_first, time, workoutid, altitude from (
            select altitude - lag(altitude) over (partition by workoutid order by time) as alt_difference, time - lag(time) over (partition by workoutid order by time) as time_difference, time, workoutid from run order by time) 
        as foo) 
    as bar order by workoutid, time )
update run r1 
  set altitude_first = d1.deriv 
  from dev_list as d1 
  where d1.workoutid = r1.workoutid and d1.time = r1.time;
```
```
UPDATE 12493175
```


##### (Re)Generate second derivative
```
with dev_list as (
    select round((alt_difference / time_difference),5) as deriv, time, workoutid from ( 
        select alt_difference, case when time_difference = 0 then 1 else time_difference end as time_difference, time, workoutid from (
            select altitude_first - lag(altitude_first) over (partition by workoutid order by time) as alt_difference, time - lag(time) over (partition by workoutid order by time) as time_difference, time, workoutid from run order by time) 
        as foo) 
    as bar order by workoutid, time )
update run r1 
  set altitude_first = d1.deriv 
  from dev_list as d1 
  where d1.workoutid = r1.workoutid and d1.time = r1.time;
```
```
UPDATE 12493175

```


#### Vacuum
```
vacumm full run;
```

## Elapsed distance


```
with elap_dist as (select time,
       r1.workoutid,
       (row_number() over (partition by r1.workoutid order by time) - 1) * (distance / (series_length - 1)) as elapsed_distance
       from run r1 join run_by_workout r2 on (r1.workoutid = r2.workoutid) order by time)   
update run as r1 
  set elapsed_distance = round(d1.elapsed_distance,10)
  from elap_dist as d1 
  where d1.workoutid = r1.workoutid and d1.time = r1.time;
```


## Elapsed time

```
with elap_time as (select time, workoutid, time - first_value(time) over (partition by workoutid order by time) as time_elap from run order by time)
update run as r1 
  set elapsed_time = d1.time_elap 
  from elap_time as d1 
  where r1.workoutid = d1.workoutid and r1.time = d1.time;
```


## Heart Rate

```
select * from histogram('heart_rate', 'run');
```
```
endomondo_sample=# select * from histogram('heart_rate', 'run');
 bucket |         range          |  freq   |       bar       
--------+------------------------+---------+-----------------
      1 | [-119.00000,-93.00000] |       4 | 
      2 | [-85.00000,-54.00000]  |       6 | 
      3 | [-47.00000,-25.00000]  |       5 | 
      4 | [-16.00000,11.00000]   |   20245 | 
      5 | [12.00000,44.00000]    |    1491 | 
      6 | [45.00000,77.00000]    |   37838 | 
      7 | [78.00000,109.00000]   |  361200 | *
      8 | [110.00000,142.00000]  | 3049231 | ********
      9 | [143.00000,175.00000]  | 5474364 | ***************
     10 | [176.00000,207.00000]  |  495725 | *
     11 | [208.00000,240.00000]  |   39443 | 
     12 | [241.00000,267.00000]  |    4103 | 
     13 | [275.00000,299.00000]  |       7 | 
     14 | [311.00000,332.00000]  |       4 | 
     15 | [351.00000,363.00000]  |       2 | 
     16 | [378.00000,393.00000]  |       2 | 
     17 | [413.00000,413.00000]  |       1 | 
     18 | [438.00000,468.00000]  |       3 | 
     19 | [480.00000,501.00000]  |       2 | 
     20 | [519.00000,519.00000]  |       1 | 
     21 | [535.00000,535.00000]  |       1 | 
```

##### Lots of garbage data. Heart rate should never be above 240 or below 40. Even those are uncommon. Use the same two standard deviation logic as altitude. 

##### Fix values too high
```
with dev_list as ( 
	select avg(heart_rate), stddev(heart_rate), workoutid from run group by workoutid )
update run as r1 
  set heart_rate = d1.avg + (d1.stddev * 2) 
  from dev_list as d1 
  where d1.workoutid = r1.workoutid and r1.heart_rate > d1.avg + (d1.stddev * 2);
```
```
UPDATE 77518
```


##### Fix values too low
```
with dev_list as ( 
	select avg(heart_rate), stddev(heart_rate), workoutid from run group by workoutid )
update run as r1 
  set heart_rate = d1.avg + (d1.stddev * 2) 
  from dev_list as d1 
  where d1.workoutid = r1.workoutid and r1.heart_rate < d1.avg - (d1.stddev * 2);
```
```
UPDATE 302130
```

#### Better, but still too many below 40 and above 250
```
select * from histogram('heart_rate', 'run');
```
```
 bucket |         range         |  freq   |       bar       
--------+-----------------------+---------+-----------------
      1 | [0.00000,0.00000]     |   20130 | 
      3 | [30.00000,41.00000]   |     659 | 
      4 | [42.00000,55.00000]   |    1664 | 
      5 | [55.69926,69.00000]   |    4291 | 
      6 | [70.00000,83.05090]   |   24722 | 
      7 | [83.74898,97.21050]   |   72291 | 
      8 | [97.69388,111.13301]  |  191009 | *
      9 | [111.20287,125.00000] |  592089 | ***
     10 | [125.10133,138.93941] | 1517767 | ********
     11 | [138.96311,152.84465] | 2953818 | ***************
     12 | [152.85173,166.73934] | 2536751 | *************
     13 | [166.74265,180.63194] | 1201726 | ******
     14 | [180.63728,194.52560] |  266629 | *
     15 | [194.53101,208.35552] |   55031 | 
     16 | [208.44704,222.28627] |   22529 | 
     17 | [222.36136,236.20774] |   14470 | 
     18 | [236.28783,250.07760] |    6902 | 
     19 | [250.28778,263.28305] |     813 | 
     20 | [266.16942,277.41503] |     368 | 
     21 | [277.90138,277.90138] |      19 | 
     ```

#### (Re)Generate moving heart rate average over last 25 points

```
with dev_list as (
	select time,
	       workoutid, 
	       avg(heart_rate) over (partition by workoutid order by time rows between 25 preceding and current row) as mavg 
	from run 
	order by time
)
update run r1 set heart_rate_ma_25 = d1.mavg from dev_list as d1 where d1.workoutid = r1.workoutid and d1.time = r1.time;
```

```
select * from histogram('heart_rate_ma_25', 'run');
```

```
 bucket |         range         |  freq   |       bar       
--------+-----------------------+---------+-----------------
      1 | [0.00000,9.50000]     |   20107 | 
      2 | [14.34615,24.15385]   |       3 | 
      3 | [29.03846,41.68476]   |     592 | 
      4 | [41.68862,55.57692]   |    1576 | 
      5 | [55.61538,69.46350]   |    3097 | 
      6 | [69.48053,83.36990]   |   17904 | 
      7 | [83.37078,97.26501]   |   57552 | 
      8 | [97.26656,111.16016]  |  151287 | *
      9 | [111.16070,125.05547] |  483707 | **
     10 | [125.05569,138.95065] | 1595715 | ********
     11 | [138.95087,152.84570] | 3121216 | ***************
     12 | [152.84576,166.74078] | 2609377 | *************
     13 | [166.74088,180.63563] | 1110636 | *****
     14 | [180.63607,194.53060] |  225084 | *
     15 | [194.53138,208.42582] |   46382 | 
     16 | [208.42782,222.31869] |   20809 | 
     17 | [222.32689,236.21502] |   13302 | 
     18 | [236.21927,250.07760] |    4909 | 
     19 | [250.12884,263.55833] |     311 | 
     20 | [264.79984,275.60195] |     100 | 
     21 | [277.90138,277.90138] |      12 | 
```